# Test Inference Server Sagemaker Object Locally

In [1]:
from sagemaker import Session, get_execution_role
from sagemaker.predictor import Predictor
from sagemaker.model import Model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from time import gmtime, strftime

import json
import os

In [2]:
def get_execution_role_in_local(sagemaker_session):
    role = sagemaker_session.boto_session.client("iam").get_role(
        RoleName="AmazonSageMaker-ExecutionRole-20230105T181131"
    )["Role"]["Arn"]
    return role

In [3]:
def load_env_variables(*env_files):
    env_vars = dict()
    get_values = lambda x: [(x.split("=")[0], x.split("=")[1])]
    for env_file in env_files:
        with open(env_file, "r") as file:
            env_vars.update(
                dict(
                    [
                        (key.strip(), value.strip()) 
                        for line in file.readlines()
                        for key, value in get_values(line)
                    ]
                )
            )
    return env_vars

In [4]:
def create_container_registry(ecr_client, repository_name, account_id):
    try:
        repository_info = ecr_client.create_repository(
            repositoryName=repository_name,
            tags=[
                {
                    "Key": "Test",
                    "Value": "True"
                }
            ],
            encryptionConfiguration={
                'encryptionType': 'AES256'
            }
        )["repository"]
    except ecr_client.exceptions.RepositoryAlreadyExistsException:
        print("repository already exists!")
        repository_info = ecr_client.describe_repositories(
            registryId=account_id,
            repositoryNames=[repository_name]
        )["repositories"][0]
    return repository_info

In [5]:
sagemaker_session = Session()
try:
    sagemaker_role = get_execution_role()
except:
    sagemaker_role = get_execution_role_in_local(sagemaker_session)

## Configurations

In [6]:
def get_configurations(stage = "staging"):
    environment = load_env_variables("../vars.env", f"../vars.{stage}.env")
    return environment

In [7]:
def set_environ_temporal_variables(**variables):
    for name, value in variables.items():
        os.environ[name] = value

In [8]:
# I've already create a vpc configuration that is able to connect to koombea db
def get_koombea_db_vpc_conf(ec2_client, account_id, security_group_name):
    # Get subnets
    subnets = ec2_client.describe_subnets(
        Filters=[
            {
                'Name':'owner-id',
                'Values':[account_id]
            }
        ]
    )
    # choose just the private subnets routing to the NateGateway
    subnets_ids = [subnets_["SubnetId"]
                   for subnets_ in subnets["Subnets"]
                   if "Tags" in subnets_.keys() and 'sm' == subnets_["Tags"][0]["Value"].split("-")[0] and "p" in subnets_["Tags"][0]["Value"]]
    # get security groups
    security_groups = ec2_client.describe_security_groups(
        Filters=[
            {
                "Name":"owner-id",
                "Values":[account_id]
            },
            {
                "Name":"group-name",
                "Values":[security_group_name]
            }
        ]
    )
    sec_groups_ids = [sec_groups_["GroupId"] for sec_groups_ in security_groups["SecurityGroups"]]
    return {"Subnets":subnets_ids,
            "SecurityGroupIds":sec_groups_ids}

In [9]:
ec2_client = sagemaker_session.boto_session.client("ec2")
security_group_name = "launch-wizard-1"
ecr_client = sagemaker_session.boto_session.client("ecr")
account_id = sagemaker_session.account_id()
aws_region = sagemaker_session.boto_region_name
repository_name = "koombea-blogs-serve-component"
docker_compose_service_name = "koombea_blogs_serve_component"
docker_image_name = "koombea_blogs_serve_{}".format(docker_compose_service_name)
stage = "staging"
environment = get_configurations(stage)
# create or get repository info
repository_info = create_container_registry(ecr_client, repository_name, account_id)
repository_uri = repository_info["repositoryUri"]
# get vpc configuration
vpc_config = get_koombea_db_vpc_conf(ec2_client, account_id, security_group_name)

repository already exists!


In [10]:
set_environ_temporal_variables(
    account_id=account_id,
    aws_region=aws_region,
    docker_compose_service_name=docker_compose_service_name,
    docker_image_name=docker_image_name,
    repository_uri=repository_uri
)

# Push container to ecr

In [11]:
%%writefile ../scripts/build_and_push_ecr.sh
echo "loging to aws ecr"
aws ecr get-login-password --region ${aws_region} | docker login --username AWS --password-stdin ${account_id}.dkr.ecr.${aws_region}.amazonaws.com

echo "building and tagging docker container"
cd ..
docker-compose build ${docker_compose_service_name}
docker tag ${docker_image_name}:latest \
    ${repository_uri}:latest

echo "pushing container"
docker push ${repository_uri}:latest
    
echo "cleaning dockers cache"
echo y | docker system prune

Overwriting ../scripts/build_and_push_ecr.sh


In [12]:
!bash ../scripts/build_and_push_ecr.sh

loging to aws ecr
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
building and tagging docker container
Building koombea_blogs_serve_component
Sending build context to Docker daemon  334.8kB
Step 1/27 : FROM continuumio/miniconda3
 ---> 45461d36cbf1
Step 2/27 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 1d42458c2aad
Step 3/27 : RUN apt-get update && apt-get install -y --no-install-recommends     curl     gcc      mono-mcs     build-essential     nginx     ca-certificates     wget     pkg-config
 ---> Using cache
 ---> 4cd8dd321ea2
Step 4/27 : RUN cd /tmp &&     wget http://download.redis.io/redis-stable.tar.gz &&     tar xvzf redis-stable.tar.gz &&     cd redis-stable &&     make &&     make install
 ---> Using cache
 ---> 15d9fd4ab548
Step 5/27 : ENV PYTHONDONTWRITEBYTECODE 1
 ---> Using cach

## Update endpoint

In [13]:
# configurations
print(stage)
if stage == "staging":
    endpoint_name = "blogsearch-stage-dev-2020-08-11-18-01-30" # dev
elif stage == "prod":
    endpoint_name = "blogsearch-stage-prod-2020-08-11-22-52-22" # prod
# model name
model_name = 'model-blogsreco-stage-{0}-{1}'.format(stage, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
# s3 bucket name
bucket_name = sagemaker_session.default_bucket()
folder_project_name = "koombea_website_ml"
folder_models_name = "koombea_blogs_models"
if stage == "staging":
    last_training_job_name = "koombea-blogs-vector-train-2023-02-08-16-28-31-172"
elif stage == "prod":
    last_training_job_name = ""
path_to_model_tar = "output/model.tar.gz"
key_output_prefix = "{}/{}/{}/{}".format(folder_project_name, folder_models_name,
                                         last_training_job_name, path_to_model_tar)
s3_bucket_model_name = "s3://{}/{}".format(bucket_name, key_output_prefix)
# instance config
instance_type = "ml.t2.medium"

staging


In [14]:
s3_bucket_model_name

's3://sagemaker-us-west-2-256305374409/koombea_website_ml/koombea_blogs_models/koombea-blogs-vector-train-2023-02-08-16-28-31-172/output/model.tar.gz'

In [15]:
model = Model(
    image_uri=repository_uri,
    model_data = s3_bucket_model_name,
    role=sagemaker_role,
    env=environment,
    name=model_name,
    vpc_config=vpc_config,
    sagemaker_session=sagemaker_session
)

In [16]:
model._create_sagemaker_model(
    instance_type=instance_type
)

### Make Predictions

In [17]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [18]:
predictor.update_endpoint(
    initial_instance_count=1,
    instance_type=instance_type,
    model_name=model.name
)

---------------------!

In [19]:
data = {
    "s": "difference between web and mobile apps", 
    "lang": "en",
    "per_page": 2,
    "page":1,
    #"term":["hi-tech", "iot"]
}
predictor.predict(data)

{'paging': {'total_count': 434,
  'total_pages': 217,
  'current_page': 1,
  'per_page': 2},
 'posts': [{'id': 3854,
   'slug': 'difference-between-mobile-apps-and-web-apps',
   'link': 'https://koombea20stg.wpengine.com/blog/difference-between-mobile-apps-and-web-apps/',
   'title': 'Mobile App Vs Web App: What’s The Difference?',
   'post_modified': 'Jan 04, 2023',
   'post_date': 'Sep 01, 2021',
   'author': 'Jonathan Tarud',
   'industry': 'App Development',
   'content_type': 'Blog',
   'image_alt': 'A user looking at his phone.',
   'image': 'https://koombea20stg.wpengine.com/wp-content/uploads/2021/04/mobile-apps-vs-web-apps-banner@2x-603x352.jpeg'},
  {'id': 6419,
   'slug': 'mobile-web-app',
   'link': 'https://koombea20stg.wpengine.com/blog/mobile-web-app/',
   'title': 'Build a Mobile Web App: Why Do You Need One?',
   'post_modified': 'Jan 04, 2023',
   'post_date': 'Oct 29, 2021',
   'author': 'Jose Gomez',
   'industry': 'App Development',
   'content_type': 'Blog',
   'i

# Test Any Endpoint

In [23]:
import boto3
import json

data = {
    "s": "difference between web and mobile apps", 
    "lang": "en",
    "per_page": 2,
    "page":1,
    #"term":["hi-tech", "iot"]
}

sagemaker_client = boto3.client("sagemaker-runtime")

In [28]:
endpoint_name = "blogsearch-stage-dev-2020-08-11-18-01-30"
response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body = json.dumps(data),
    ContentType = "application/json"
)

print(json.dumps(json.loads(response["Body"].read().decode()), indent=2))

{
  "paging": {
    "total_count": 464,
    "total_pages": 232,
    "current_page": 1,
    "per_page": 2
  },
  "posts": [
    {
      "id": 3854,
      "slug": "difference-between-mobile-apps-and-web-apps",
      "link": "https://koombea20stg.wpengine.com/blog/difference-between-mobile-apps-and-web-apps/",
      "title": "Mobile App Vs Web App: What\u2019s The Difference?",
      "post_modified": "Jan 04, 2023",
      "post_date": "Sep 01, 2021",
      "author": "Jonathan Tarud",
      "industry": "App Development",
      "content_type": "Blog",
      "image_alt": "A user looking at his phone.",
      "image": "https://koombea20stg.wpengine.com/wp-content/uploads/2021/04/mobile-apps-vs-web-apps-banner@2x-603x352.jpeg"
    },
    {
      "id": 6419,
      "slug": "mobile-web-app",
      "link": "https://koombea20stg.wpengine.com/blog/mobile-web-app/",
      "title": "Build a Mobile Web App: Why Do You Need One?",
      "post_modified": "Jan 04, 2023",
      "post_date": "Oct 29, 2021",

In [29]:
endpoint_name = "blogsearch-stage-prod-2020-08-11-22-52-22"
response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body = json.dumps(data),
    ContentType = "application/json"
)

print(json.dumps(json.loads(response["Body"].read().decode()), indent=2))

{
  "paging": {
    "total_count": 492,
    "total_pages": 246,
    "current_page": 1,
    "per_page": 2
  },
  "posts": [
    {
      "id": 3854,
      "slug": "difference-between-mobile-apps-and-web-apps",
      "link": "https://www.koombea.com/blog/difference-between-mobile-apps-and-web-apps/",
      "title": "Mobile App Vs Web App: What\u2019s The Difference?",
      "post_modified": "Feb 02, 2023",
      "post_date": "Sep 01, 2021",
      "author": "Jonathan Tarud",
      "industry": "App Development",
      "content_type": "Blog",
      "image_alt": "A user looking at his phone.",
      "image": "https://www.koombea.com/wp-content/uploads/2021/04/mobile-apps-vs-web-apps-banner@2x-603x352.jpeg"
    },
    {
      "id": 6419,
      "slug": "mobile-web-app",
      "link": "https://www.koombea.com/blog/mobile-web-app/",
      "title": "Build a Mobile Web App: Why Do You Need One?",
      "post_modified": "Feb 02, 2023",
      "post_date": "Oct 29, 2021",
      "author": "Jose Gomez",